# MCT4052 Workshop 6f: Repeated K-Fold Cross Validation

*Author: Stefano Fasciani, stefano.fasciani@imv.uio.no, Department of Musicology, University of Oslo.*

In notebook we use the [RepeatedKFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RepeatedKFold.html) and [RepeatedStratifiedKFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RepeatedStratifiedKFold.html) to further improve the estimation of the model performances via cross validation. In particular Repeated K-Fold is likely to reduce the bias in the model’s estimated performance (although it may increase the variance) because the split in multiple time is performed randomly multiple time (and at each time a cross validation is performed). This reduced the likelyhood to have a particularly favorable split in our cross validation.

Stratification is the process of rearranging the data as to ensure each fold is a good representation of the whole set. For example in a binary classification problem where each class comprises 50% of the data, it is best to arrange the data such that in every fold, each class comprises around half the instances. The repeated stratified k-fold take care of this aspect. Mind that the stratification process make sense and it can be applied only to classification problems, while the repeated k-fold object works with both classification and regression problems. 

In this notebook we demonstrate how to apply repeated k-fold cross validation to a regression task, and how to apply stratified repeated k-fold to a classification task.


In [1]:
import numpy as np
import pandas as pd
import librosa
import sklearn
import os

### 1. Regression task with repeated k-fold cross validation

In [2]:
#loading files and computing features
sr = 22050

def extract_features_target(filename, sr):
    
    signal, dummy = librosa.load(filename, sr=sr, mono=True)
    output = librosa.feature.melspectrogram(y=signal, n_mels=25)
    output = output.flatten()
    
    #preparing the output array
    target = np.zeros((1,2))
    target[0,0] = np.mean(librosa.feature.rms(y=signal))
    target[0,1] = np.mean(librosa.feature.spectral_flatness(y=signal))
    
    return output, target

filenames = os.listdir('./data/examples3')
features = np.zeros((len(filenames),4325))
target = np.zeros((len(filenames),2))

for i in range(len(filenames)):
    features[i,:], target[i,:] = extract_features_target('./data/examples3/'+filenames[i], sr=sr)

print('Done!')

Done!


In [3]:
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPRegressor
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedKFold


#creating pipeline
pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('dim_red', PCA(n_components = 5)),
        ('classifier', MLPRegressor(hidden_layer_sizes=(12,8,4), max_iter=2000, activation='tanh'))
        ])

#creating the repeated k-fold, use random_state to get repeatable results
#with n_splits=5 we partition the data into 5 splits of 20% and use 4 for trainign and 1 for testing.
#the n_repeats indicates how many times the k-fold has to be repeated
rkf = RepeatedKFold(n_splits=5, n_repeats=10)

#initializing the cross validator with pipe, features, target, scores, and kfold object
scores = sklearn.model_selection.cross_validate(pipe, features, target, cv=rkf, scoring=('r2', 'neg_mean_squared_error'),return_train_score=True)

print(scores,'\n')

print('MSE mean and variance', np.mean(scores['test_neg_mean_squared_error']),np.var(scores['test_neg_mean_squared_error']),'\n')
print('R2 mean and variance', np.mean(scores['test_r2']),np.var(scores['test_r2']),'\n')


{'fit_time': array([0.10154796, 0.10232663, 0.15194821, 0.11705613, 0.14870119,
       0.08661199, 0.14207625, 0.09549093, 0.16779613, 0.1067152 ,
       0.10799623, 0.09253407, 0.10906816, 0.18411088, 0.10292816,
       0.06886029, 0.09984684, 0.11520696, 0.09330273, 0.12761188,
       0.07633376, 0.12302423, 0.14232206, 0.24080014, 0.09807301,
       0.17648816, 0.09733987, 0.10529518, 0.10580397, 0.139189  ,
       0.11333609, 0.16447902, 0.0984199 , 0.15436387, 0.12027693,
       0.11258197, 0.09731007, 0.0964129 , 0.1009872 , 0.10322523,
       0.1542511 , 0.10777211, 0.16953707, 0.14464116, 0.12421012,
       0.166857  , 0.07628417, 0.14685488, 0.23637509, 0.18907905]), 'score_time': array([0.00286078, 0.00272632, 0.00222898, 0.00242877, 0.00230598,
       0.00284219, 0.00230789, 0.00283003, 0.00248694, 0.00279498,
       0.003232  , 0.00290608, 0.00294304, 0.00227809, 0.00270081,
       0.00269485, 0.00252128, 0.00259614, 0.00260019, 0.00253797,
       0.00269794, 0.0028069 , 0.

### 3. Classification task with stratified repeated k-fold cross validation

In [4]:
#loading files and extracting features
metadata = pd.read_csv('./data/examples4/meta.csv')
classes = list(metadata.label.unique())
print('There are',len(classes),'different classes:',classes)

sr = 22050

def extract_features(filename, sr):
    signal, dummy = librosa.load(filename, sr=sr, mono=True)
    output = np.mean(librosa.feature.mfcc(y=signal, n_mfcc=20), axis=1)
    return output

print('number of files in database',len(metadata.index))
features = np.zeros((len(metadata.index),20))
labels = np.zeros((len(metadata.index)))

for i, row in metadata.iterrows():
    features[i,:] = extract_features('./data/examples4/'+row['filename'], sr=sr)
    labels[i] = (classes.index(row['label']))

print('Done!')

There are 5 different classes: ['cello', 'guitar', 'clarinet', 'flute', 'harmonica']
number of files in database 60
Done!


In [5]:
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedStratifiedKFold

#creating pipeline
pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('dim_red', PCA(n_components = 10)),
        ('classifier', MLPClassifier(hidden_layer_sizes=(20,5), max_iter=10000, activation='relu'))
        ])

#creating the repeated stratified k-fold, use random_state to get repeatable results
#with n_splits=5 we partition the data into 5 splits of 20% and use 4 for trainign and 1 for testing.
#the n_repeats indicates how many times the k-fold has to be repeated
rkf = RepeatedStratifiedKFold(n_splits=5, n_repeats=10)

#initializing and running the cross validator with pipe, features, labels, scores, and kfold object
scores = sklearn.model_selection.cross_validate(pipe, features, labels, cv=rkf, scoring=('f1_macro', 'accuracy'),return_train_score=True)

print(scores,'\n')
print('Accuracy mean and variance', np.mean(scores['test_accuracy']),np.var(scores['test_accuracy']),'\n')
print('F1 macro mean and variance', np.mean(scores['test_f1_macro']),np.var(scores['test_f1_macro']),'\n')


{'fit_time': array([0.34852886, 0.48296809, 0.25097084, 0.23481441, 0.26952696,
       0.23894501, 0.32500887, 0.28511   , 0.22001195, 0.28299093,
       0.50000167, 0.31847572, 0.28933501, 0.26955795, 0.27311182,
       0.29453897, 0.25470424, 0.21297622, 0.28232121, 0.23591304,
       0.27149081, 0.28396797, 0.26231289, 0.31240988, 0.23313808,
       0.28538108, 0.29141808, 0.28191113, 0.28739095, 0.28513503,
       0.24892783, 0.29902196, 0.30431509, 0.25590205, 0.25323606,
       0.38557696, 0.31396508, 0.20800591, 0.22099209, 0.25019383,
       0.26580811, 0.23003507, 0.2738111 , 0.27208996, 0.39080882,
       0.2298491 , 0.32316279, 0.25985098, 0.35246992, 0.23148084]), 'score_time': array([0.00267482, 0.00253987, 0.00216913, 0.00220799, 0.00216889,
       0.00302529, 0.00207806, 0.00231504, 0.00221515, 0.00251794,
       0.00212717, 0.00212121, 0.00223207, 0.00232124, 0.00210285,
       0.00233817, 0.00213671, 0.00248098, 0.00215292, 0.00207305,
       0.0024693 , 0.00220609, 0.

### 4. Follow up activity

Apply the repeated k fold and stretified repeated k fold to classification and regression on ML applications you previously developed using both your databases. It is recommended to use pipelines.